<a href="https://colab.research.google.com/github/Kingnamji/DACON_BIT_TRADER/blob/main/Bit_Trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이콘 인공지능 비트 트레이더 경진대회

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import sys, os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import seaborn as sns


# tools to plot a graph

%matplotlib inline
import matplotlib.pyplot as plt


In [4]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/데이콘')

Read datasets from csv file

In [5]:
train_x_df = pd.read_csv('train_x_df.csv')
train_y_df = pd.read_csv('train_y_df.csv')
test_x_df = pd.read_csv('test_x_df.csv')

train_x_df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
0,0,0,7,1.010004,1.010003924369812,1.009612,1.010004,8.382875e+05,43160.632812,451.157288,7.326834e+05,37725.183594
1,0,1,7,1.009808,1.0098077058792114,1.009808,1.009808,1.622420e+05,8352.220703,39.231071,0.000000e+00,0.000000
2,0,2,7,1.009808,1.010200023651123,1.009808,1.010200,1.664967e+04,857.377808,58.846603,1.664967e+04,857.377808
3,0,3,7,1.010200,1.0111808776855469,1.010200,1.011181,2.586971e+06,133310.343750,431.541779,2.189147e+06,112811.046875
4,0,4,7,1.010985,1.0109846591949463,1.010200,1.010200,1.129996e+06,58216.867188,176.539810,0.000000e+00,0.000000


In [6]:
train_x_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10159560 entries, 0 to 10159559
Data columns (total 12 columns):
 #   Column       Dtype  
---  ------       -----  
 0   sample_id    int64  
 1   time         int64  
 2   coin_index   int64  
 3   open         float64
 4   high         object 
 5   low          float64
 6   close        float64
 7   volume       float64
 8   quote_av     float64
 9   trades       float64
 10  tb_base_av   float64
 11  tb_quote_av  float64
dtypes: float64(8), int64(3), object(1)
memory usage: 930.1+ MB


In [7]:
train_x_df.isnull().sum()

sample_id      0
time           0
coin_index     0
open           0
high           0
low            0
close          0
volume         0
quote_av       0
trades         0
tb_base_av     0
tb_quote_av    0
dtype: int64

In [8]:
''' open, low, volume 속성의 값이 0인 행 삭제 '''

indexNames = train_x_df[(train_x_df['volume'] == 0)].index
train_x_df.drop(indexNames, inplace = True)

In [9]:
train_x_df.shape

(9633512, 12)

low의 type은 float64임에 반해, high의 type은 object이다.

high에서 값의 일부가 이상한 값이 나온 듯 하다. 전처리에서 해줘야겠다.

In [10]:
print(train_x_df.shape, train_y_df.shape)

(9633512, 12) (883440, 12)


In [11]:
def df2d_to_array3d(df_2d):
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d

In [13]:
''' 이런 ! '''
train_x_array = df2d_to_array3d(train_x_df)
train_y_array = df2d_to_array3d(train_y_df)
test_x_array = df2d_to_array3d(test_x_df)

ValueError: ignored

In [ ]:
print(f'''
train_x_array {train_x_array.shape}
train_y_array {train_y_array.shape}
test_x_array {test_x_array.shape}
''')

모델의 데이터 개수가 너무 많다 ... 그래서 이상한 값, NaN 값 등이 있는 것은 과감히 버리기로 !

In [ ]:
''' 데이터에 0 값 많은거(쓸모 없는거 !)는 제거 '''
